# 房價預測
> https://www.kaggle.com/c/house-prices-advanced-regression-techniques

## 01. 讀取dataset

In [1]:
import numpy as np
import pandas as pd

DF = pd.read_csv('train.csv')
DF.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## 02. 前處理
### 檢查空值
列出有空值的column名稱

In [8]:
DF.columns[DF.isna().any()].tolist()

['LotFrontage',
 'Alley',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

列出缺值數目

In [3]:
NA_columns = DF.columns[DF.isna().any()].tolist()
DF[NA_columns].isnull().sum()

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

### 選定特徵與標籤

In [2]:
NA_columns = DF.columns[DF.isna().any()].tolist()
train_feature = DF.drop(NA_columns, axis=1)
train_label = DF['SalePrice'].copy()

## 03. One Hot Encoding

In [16]:
train_feature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 62 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotArea          1460 non-null int64
Street           1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
ExterQual        1460 non-

先使用數值型態的feature

In [17]:
train_feature.select_dtypes(include=[np.number]).columns.tolist()

['Id',
 'MSSubClass',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'SalePrice']

In [3]:
numeric_DF = train_feature.select_dtypes(include=[np.number]).columns.tolist()
train_feature_num = train_feature[numeric_DF].copy()

In [4]:
# 避免overflow/ inf
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
column_names = list(train_feature_num)
train_feature_num = scaler.fit_transform(train_feature_num)
# 從 ndarray轉回dataframe
train_feature_num = pd.DataFrame(train_feature_num, columns=column_names)
train_feature_num.head()

C:\Users\Wade\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Wade\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,-1.730865,0.073375,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.575425,-0.288653,-0.944591,...,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,0.138777,0.347273
1,-1.728492,-0.872563,-0.091886,-0.071836,2.179628,0.156734,-0.429577,1.171992,-0.288653,-0.641228,...,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-0.489110,-0.614439,0.007288
2,-1.726120,0.073375,0.073480,0.651479,-0.517200,0.984752,0.830215,0.092907,-0.288653,-0.301643,...,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.990891,0.138777,0.536154
3,-1.723747,0.309859,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.499274,-0.288653,-0.061670,...,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,-1.367655,-0.515281
4,-1.721374,0.073375,0.375148,1.374795,-0.517200,0.951632,0.733308,0.463568,-0.288653,-0.174865,...,0.780197,0.563760,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,2.100892,0.138777,0.869843


## 05. Algorithm/ Model
### Linear Regression with SGD approach
![](https://image.slidesharecdn.com/anoverviewofgradientdescentoptimizationalgorithms-170414055411/95/an-overview-of-gradient-descent-optimization-algorithms-6-638.jpg?cb=1492149859)

In [8]:
weights = np.ones(len(train_feature_num.columns))

bias = 0

learningRate = 0.001

Iterations = 10
# 紀錄loss變化
loss_record = []

In [9]:
# 訓練參數
def train_lr(data, labels, iterations):
    global loss_record
    for numIter in range(Iterations):
        # batch size為1, 每筆資料都更新參數
        for i in range(len(data.index)):
            # 資料
            input = data.iloc[i]
            
            global weights, bias
            # 預測價格
            predict_price = np.dot(input, weights) + bias
#             print(predict_price)
            
            # 預測值與實際值間的誤差
            loss = predict_price - labels[i]
#             print(loss)
            
            # 偏微分, 計算梯度
            gradient = np.dot(input.T, loss)
#             print(gradient)
            
            # 參數更新
            weights = weights - (1/len(data.index))*learningRate*gradient
#             weights = weights - learningRate*gradient
            # bias更新
            bias = bias - (1/len(data.index))*learningRate*loss
        
        # 在最後一筆資料時計算現在的 weight的 MSE
        all_predict = weights*data +bias
        # 同一 row做 sum
        all_loss =  all_predict.sum(axis=1) -labels
        MSE = np.mean(all_loss**2)
        # 紀錄loss
        loss_record.append(MSE)

In [10]:
train_lr(train_feature_num, train_label, Iterations)
loss_record

[36714644075.54121,
 34474850707.64771,
 32318071372.20907,
 30244050237.397137,
 28252532235.613537,
 26343263059.388336,
 24515989157.3215,
 22770457730.068275,
 21106416726.367012,
 19523614839.10938]

In [26]:
# remove (1/len(data.index)), loss小一點
train_lr(train_feature_num, train_label, Iterations)
loss_record

[16870870254.0167,
 14721720401.203047,
 14410981272.466742,
 14465882990.790627,
 14583453764.575897,
 14697298801.516237,
 14794477087.197018,
 14874380155.016512,
 14939107869.5738,
 14991140644.015417]

## 06. Predict

In [ ]:
def predict(data):
    prediction = []
     for i in range(len(data.index)):

In [ ]:
DF_submit = pd.DataFrame(
    {'Id': answer['Id'],
     'SalePrice': y_pred
    })

DF_submit.to_csv('submit_lr.csv', index=False)
DF_submit.head()